In [1]:
import torch
import non_local_boxes

# Sugar coating for reloading
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Implementation of `rho(P)`

In [23]:
# def BG15_form(P): # P is a 4x4 box, in the usual form
#     P = non_local_boxes.utils.matrix_to_tensor(P)
#     Q = torch.zeros(4,4)
#     for i in range(4):
#         a, x = i//2, i%2
#         for j in range(4):
#             b, y = j//2, j%2
#             Q[i,j] = P[a, b, x, y]
#     return Q

def BG15_form(P): # P is a 4x4 box, in the usual form
    P = non_local_boxes.utils.matrix_to_tensor(P)
    Q = torch.zeros(4,4)
    for i in range(4):
        a, x = i//2, i%2
        for j in range(2):
            # b, y = j, x
            Q[i, 2*j + x] = P[a, j, x, x]
    return Q

In [30]:
def marginals(P):  # P is a 4x4 matrix, in the BS15 form
    # P = BG15_form(P)
    e = torch.ones(4)
    pX = torch.tensordot(P, e, dims=1)
    pY = torch.tensordot(torch.t(P), e, dims=1)
    return pX, pY

def marginals_diag(P):  # P is a 4x4 matrix, in the BS15 form
    pX, pY = marginals(P)
    P_X, P_Y = torch.diag_embed(pX), torch.diag_embed(pY)
    return P_X, P_Y

In [39]:
def P_tilda(P):  # P is a 4x4 matrix, in the usual form
    P = BG15_form(P)
    pX, pY = marginals(P)
    A, B = torch.diag_embed(torch.pow(pX, -0.5)), torch.diag_embed(torch.pow(pY, -0.5))
    return torch.tensordot(torch.tensordot(A, P, dims=1), B, dims=1)

In [61]:
def rho(P):   # P is a 4x4 matrix, in the usual form
    sing_val = torch.linalg.svdvals(P_tilda(P))
    return round(float(sing_val[1]), 6)

# Compute $\rho$ of somes boxes

In [ ]:
PR = non_local_boxes.utils.PR
SR = non_local_boxes.utils.SR
I = non_local_boxes.utils.I

In [62]:
print(rho(PR), rho(SR), rho(I))

1.0 1.0 1.0
